# Fine-tune Qwen3 on Amazon SageMaker

In [1]:
%store -r

In [2]:
print(f"test_model_id : {test_model_id}")
print(f"bucket : {bucket}")
print(f"model_weight_path : {model_weight_path}")
print(f"training_input_path : {training_input_path}")
# print(f"test_input_path : {test_input_path}")
print(f"local_training_input_path : {local_training_input_path}")
# print(f"local_test_input_path : {local_test_input_path}")
print(f"registered_model : {registered_model}")

test_model_id : Qwen/Qwen3-4B
bucket : sagemaker-us-west-2-322537213286
model_weight_path : s3://sagemaker-us-west-2-322537213286/checkpoints/qwen3-4b
training_input_path : s3://sagemaker-us-west-2-322537213286/korean-openthoughts-114k-normalized/train/train_dataset.json
local_training_input_path : /home/ec2-user/SageMaker/TRAINING/qwen3-on-sagemaker/dataset/train
registered_model : qwen3-4b


In [3]:
import sagemaker
from pathlib import Path
from time import strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/05/25 09:44:00] INFO     Found credentials from IAM Role:                                   ]8;id=924007;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=751819;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=189522;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=432117;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=922759;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=436893;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [4]:
sagemaker.__version__

'2.243.3'

## 2. 모델 fine-tuning을 위한 파라미터 설정

이제 모델을 fine-tuning할 준비가 되었습니다. `trl`의 [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)를 사용하여 모델을 fine-tuning하겠습니다. SFTTrainer는 오픈 LLM을 지도 학습 방식으로 fine-tuning하는 것을 간소화합니다. SFTTrainer는 `transformers`의 `Trainer` 클래스의 하위 클래스입니다. 데이터셋을 디스크에서 로드하고, 모델과 토크나이저를 준비하고 훈련을 시작하는 스크립트 [sm_qlora_trainer.py](./src/sm_qlora_trainer.py)를 준비했습니다. 이 스크립트는 `trl`의 [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)를 사용하여 모델을 fine-tuning하며 다음 기능을 지원합니다:
`yaml` 파일은 데이터셋과 유사하게 Amazon SageMaker에 업로드되고 제공됩니다. 이 설정 파일을 `qwen3-4b.yaml`로 저장하고 S3에 업로드합니다.

In [5]:
!mkdir -p src/configs

In [6]:
%%writefile src/configs/qwen3-4b.yaml
# 스크립트 기본 매개변수
model_name_or_path: "/opt/ml/input/data/model_weight"
train_dataset_path: "/opt/ml/input/data/training"
output_dir: "/opt/ml/checkpoints"
tokenizers_parallelism: "false"

# 모델 설정 - 메모리 최적화
model:
  load_in_4bit: true
  bnb_4bit_use_double_quant: true  # 이중 양자화 활성화
  bnb_4bit_quant_type: "nf4"
  use_bf16: false  # fp16 사용 (메모리 효율적)
  trust_remote_code: true
  low_cpu_mem_usage: true
  use_cache: false  # 캐시 비활성화로 메모리 절약
  offload_folder: "offload"  # 디스크 오프로딩 설정
  offload_state_dict: true  # 상태 딕셔너리 오프로딩

# 토크나이저 설정
tokenizer:
  trust_remote_code: true
  use_fast: true
  padding_side: "right"

# LoRA 설정 - 메모리 최적화
lora:
  lora_alpha: 16
  lora_dropout: 0.05
  lora_r: 64  # r 값 감소로 메모리 사용량 감소
  bias: "none"
  target_modules:
    - "q_proj"
    - "k_proj"
    - "v_proj"
    - "o_proj"
    - "gate_proj"
    - "up_proj"
    - "down_proj"

# 데이터 설정 - 메모리 최적화
data:
  train_path: "train_dataset.json"
  text_column: "text"
  max_seq_length: 2048
  padding: false  # 동적 패딩 사용
  truncation: true

# 데이터셋 처리 설정 - 메모리 최적화
dataset:
  preprocessing_batch_size: 50  # 작은 배치 크기로 처리
  num_proc: 1
  streaming: false  # 필요시 true로 설정하여 스트리밍 활성화

# 데이터 콜레이터 설정
data_collator:
  mlm: false
  pad_to_multiple_of: 8

# 학습 설정 - 메모리 최적화
training:
  per_device_train_batch_size: 1  # 배치 크기 감소
  gradient_accumulation_steps: 8  # 증가하여 효과적인 배치 크기 유지
  learning_rate: 2.0e-3
  num_train_epochs: 5
  logging_steps: 10
  warmup_steps: 10
  optim: "adamw_torch_fused"  # 최적화된 옵티마이저
  group_by_length: true  # 길이별 그룹화로 패딩 최소화
  save_strategy: "steps"
  save_steps: 500
  save_total_limit: 1  # 저장 모델 수 감소
  seed: 42
  dataloader_num_workers: 0  # 워커 수 감소
  report_to: "none"  # 보고 비활성화
  ddp_find_unused_parameters: false
  gradient_checkpointing: true  # 그래디언트 체크포인팅 활성화
  max_grad_norm: 1.0

Overwriting src/configs/qwen3-4b.yaml


In [7]:
# from sagemaker.huggingface import HuggingFace
# import torch

training_hyperparameters={}

## Create SageMaker Training Job

SageMaker 학습 작업을 생성하기 위해서는 `HuggingFace` Estimator가 필요합니다. Estimator는 Amazon SageMaker의 end-to-end 학습 및 배포 작업을 처리합니다. Estimator는 인프라 사용을 관리합니다. Amazon SageMaker는 필요한 모든 ec2 인스턴스를 시작하고 관리하며, 적절한 huggingface 컨테이너를 제공하고, 제공된 스크립트를 업로드하고 S3 버킷의 데이터를 컨테이너의 `/opt/ml/input/data`로 다운로드합니다. 그런 다음 학습 작업을 시작합니다.

> Note: 사용자 정의 학습 스크립트를 사용하는 경우 `source_dir`에 `requirements.txt`를 포함해야 합니다. 전체 리포지토리를 클론하는 것을 권장합니다.

스크립트 실행에 `torchrun`을 사용하려면 Estimator에서 `distribution` 파라미터를 정의하고 `{"torch_distributed": {"enabled": True}}`로 설정하기만 하면 됩니다. 이렇게 하면 SageMaker가 다음과 같이 학습 작업을 실행합니다:

```python
torchrun --nnodes 2 --nproc_per_node 8 --master_addr algo-1 --master_port 7777 --node_rank 1 sm_qlora_trainer.py --config /opt/ml/input/data/config/config.yaml
```
아래의 HuggingFace 설정은 1x A10 GPU가 있는 1x ml.g5.2xlarge에서 학습 작업을 시작합니다. SageMaker의 놀라운 점은 instance_count를 수정하여 쉽게 ml.p4d.24xlarge 또는 2x ml.p4d.24xlarge로 확장할 수 있다는 것입니다. SageMaker가 나머지를 처리해줍니다.

In [8]:
instance_type = 'ml.g5.2xlarge'
# instance_type = 'ml.p4d.24xlarge'
# instance_type = 'ml.p5.48xlarge'
# instance_type = 'local_gpu'
instance_count = 1
max_run = 72*60*60

In [9]:
local_model_weight_path = f"{Path.cwd()}/{registered_model}"
local_model_weight_path

'/home/ec2-user/SageMaker/TRAINING/qwen3-on-sagemaker/qwen3-4b'

In [10]:
if instance_type =='local_gpu':
    import os
    from sagemaker.local import LocalSession

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    training = f"file://{local_training_input_path}"
    # test = f"file://{local_test_input_path}"
    model_weight = f"file://{local_model_weight_path}"
else:
    sagemaker_session = sagemaker.Session()
    training = training_input_path
    # test = test_input_path
    model_weight = model_weight_path

training, model_weight

[05/05/25 09:44:01] INFO     Found credentials from IAM Role:                                   ]8;id=30380;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=172392;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

('s3://sagemaker-us-west-2-322537213286/korean-openthoughts-114k-normalized/train/train_dataset.json',
 's3://sagemaker-us-west-2-322537213286/checkpoints/qwen3-4b')

In [11]:
from sagemaker.pytorch import PyTorch
import time
# define Training Job Name 
job_name = f'huggingface-{registered_model}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# distribution={ "pytorchddp": { "enabled": True } }  # mpirun, activates SMDDP AllReduce OR AllGather
# distribution={"mpi": {"enabled": True}}
distribution={
    "torch_distributed": {
        "enabled": True,
        # "NCCL_DEBUG":"INFO"
        # "mpi": "-verbose -x NCCL_DEBUG=INFO"
    }
}  # torchrun, activates SMDDP AllGather
# distribution={ "smdistributed": { "dataparallel": { "enabled": True } } }  # mpirun, activates SMDDP AllReduce OR AllGather

environment={
    "NCCL_DEBUG" : "INFO", 
    "SM_LOG_LEVEL": "10",
}

training_hyperparameters["config"] = "/opt/ml/code/configs/qwen3-4b.yaml"
    
estimator = PyTorch(
                    entry_point='sm_lora_trainer.py',
                    source_dir=f'{Path.cwd()}/src',
                    role=role,
                    # image_uri=image_uri,
                    framework_version='2.3.0',
                    py_version='py311',
                    instance_count=instance_count,
                    instance_type=instance_type,
                    distribution=distribution,
                    disable_profiler=True,
                    debugger_hook_config=False,
                    max_run=max_run,
                    hyperparameters=training_hyperparameters,
                    sagemaker_session=sagemaker_session,
                    # enable_remote_debug=True,
                    # keep_alive_period_in_seconds=1200,
                    # input_mode='FastFile'
                    # max_wait=max_run,
                    # use_spot_instances=True,
                    # subnets=['subnet-090e278f3622051c4'],
                    # security_group_ids=['sg-05baa06337a188842'],
                    max_retry_attempts=30,
                    environment=environment,
                   )

                    INFO     Found credentials from IAM Role:                                   ]8;id=808254;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=328994;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [12]:
!sudo rm -rf src/core.*

In [13]:
current_time = strftime("%m%d-%H%M%s")
i_type = instance_type.replace('.','-')
job_name = f'{registered_model}-{i_type}-{instance_count}-{current_time}'


if instance_type =='local_gpu':
    estimator.checkpoint_s3_uri = None
else:
    estimator.checkpoint_s3_uri = f's3://{bucket}/checkpoint/{test_model_id}/{job_name}'
    
    
estimator.fit(
    inputs={
        'training': training,
        'model_weight' : model_weight
    }, 
    job_name=job_name,
    wait=False
)

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=272963;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=156410;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[05/05/25 09:44:02] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=403348;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=567929;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=871257;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=435907;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241                                          

In [14]:
sagemaker_session = sagemaker.Session()
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2025-05-05 09:44:02 Starting - Starting the training job
2025-05-05 09:44:02 Pending - Training job waiting for capacity...............
2025-05-05 09:46:28 Pending - Preparing the instances for training...
2025-05-05 09:46:56 Downloading - Downloading input data.........
2025-05-05 09:48:11 Downloading - Downloading the training image............
2025-05-05 09:50:17 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 530.30.02
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
/opt/conda/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithm

## PEFT 모델 추론 하기

In [15]:
import sagemaker
sagemaker_session = sagemaker.Session()
train_result = sagemaker_session.describe_training_job(job_name=job_name)

In [16]:
checkpoint_s3uri = train_result['CheckpointConfig']['S3Uri']
checkpoint_s3uri

's3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241'

In [17]:
!aws s3 ls $checkpoint_s3uri/

                           PRE checkpoint-130/
2025-05-05 10:03:16       5105 README.md
2025-05-05 10:03:17        864 adapter_config.json
2025-05-05 10:03:17  528550256 adapter_model.safetensors
2025-05-05 10:03:17        707 added_tokens.json
2025-05-05 10:03:17    1671853 merges.txt
2025-05-05 10:03:17        613 special_tokens_map.json
2025-05-05 10:03:17   11422934 tokenizer.json
2025-05-05 10:03:17       9706 tokenizer_config.json
2025-05-05 10:03:17    2776833 vocab.json


In [18]:
output_dir = './checkpoints'

In [19]:
!rm -rf $output_dir
!aws s3 sync $checkpoint_s3uri $output_dir

download: s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241/README.md to checkpoints/README.md
download: s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241/checkpoint-130/README.md to checkpoints/checkpoint-130/README.md
download: s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241/adapter_config.json to checkpoints/adapter_config.json
download: s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241/checkpoint-130/adapter_config.json to checkpoints/checkpoint-130/adapter_config.json
download: s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2xlarge-1-0505-09441746438241/checkpoint-130/added_tokens.json to checkpoints/checkpoint-130/added_tokens.json
download: s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/qwen3-4b-ml-g5-2

In [20]:
!rm -rf $output_dir/checkpoint-*
!rm -rf $output_dir/compressed_model
!rm -rf $output_dir/runs

In [21]:
local_model_weight_path=f'{Path.cwd()}/{registered_model}'

In [22]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
peft_model_id = output_dir

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    local_model_weight_path,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
peft_model = PeftModel.from_pretrained(base_model, peft_model_id)
peft_model = peft_model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
merged_save_dir = "merged_model"
peft_model.save_pretrained(merged_save_dir, safe_serialization=True, max_shard_size="2GB")

In [24]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(local_model_weight_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.save_pretrained(merged_save_dir)

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/vocab.json',
 'merged_model/merges.txt',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [25]:
import torch
torch.cuda.empty_cache()
device = torch.cuda.current_device()

In [26]:
inference_prompt_style = """너는 reasoning, analysis, problem-solving에 advanced knowledge를 갖춘 AI Assistant입니다.
    <question> 질문에 가장 적절한 답변을 작성하세요. 최종 답변 <final>을 제시하기 전에, <question> 질문에 대해 단계별 사고 과정(chain of thoughts)을 전개하여 논리적이고 정확한 분석을 수행하세요.
    
    <question>
    {}
    </question>
    ### 주의사항:
    - 불필요한 인사말이나 서두, input은 생략하고, 바로 <response> 부터 작성해주세요.
    - 질문과 답변을 반복하지 마세요
    - 단계별 사고 과정은 충분히 상세하게 작성하되, 최종 답변은 간결하게 정리하세요
    

    
    ### 응답 형식:
    <think>
        ### THINKING
        [여기에 한국어로 단계별 사고 과정을 상세히 기술하세요. 문제를 분석하고, 가능한 접근법을 검토하며, 논리적 추론을 통해 결론에 도달하는 과정을 보여주세요.]
    </think>
    <final>
        ### FINAL-ANSWER
        [THINKING에서 도출된 결론을 간결하고 명확하게 요약하여 한국어로 최종 답변으로 제시하세요.]
    </final>

    아래 답변입니다.
    <think>
    """

In [27]:
%%time
max_new_tokens = 1024

input_ids = tokenizer(
    [inference_prompt_style.format("서울의 유명한 관광 코스를 만들어줄래?") + tokenizer.eos_token], return_tensors="pt"
).input_ids

input_ids = input_ids.to(device)

outputs = peft_model.generate(input_ids, max_new_tokens=max_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


너는 reasoning, analysis, problem-solving에 advanced knowledge를 갖춘 AI Assistant입니다.
    <question> 질문에 가장 적절한 답변을 작성하세요. 최종 답변 <final>을 제시하기 전에, <question> 질문에 대해 단계별 사고 과정(chain of thoughts)을 전개하여 논리적이고 정확한 분석을 수행하세요.
    
    <question>
    서울의 유명한 관광 코스를 만들어줄래?
    </question>
    ### 주의사항:
    - 불필요한 인사말이나 서두, input은 생략하고, 바로 <response> 부터 작성해주세요.
    - 질문과 답변을 반복하지 마세요
    - 단계별 사고 과정은 충분히 상세하게 작성하되, 최종 답변은 간결하게 정리하세요
    

    
    ### 응답 형식:
    <think>
        ### THINKING
        [여기에 한국어로 단계별 사고 과정을 상세히 기술하세요. 문제를 분석하고, 가능한 접근법을 검토하며, 논리적 추론을 통해 결론에 도달하는 과정을 보여주세요.]
    </think>
    <final>
        ### FINAL-ANSWER
        [THINKING에서 도출된 결론을 간결하고 명확하게 요약하여 한국어로 최종 답변으로 제시하세요.]
    </final>

    아래 답변입니다.
    <think>
    
        ### THINKING
        이 질문에 답하기 위해서는 서울의 주요 관광지와 역사, 문화, 현대적인 랜드마크 등을 균형 있게 포함해야 한다. 명소, 전통 문화 관련 장소, 현대적인 관광지, 그리고 코스 내에서의 이동 편의성을 고려해야 한다. 각 관광지의 특징을 간략히 설명하면서, 서울 관광의 대표성을 갖춘 코스를 구성하는 것이 좋을 것이다.
    </think>
    <final>
        ### FINAL-ANSWER
      

In [28]:
import os
os.makedirs('shell', exist_ok=True)
compressed_model_path='/'.join(checkpoint_s3uri.split("/")[:-1]) + "/compressed_model"
compressed_model_path

's3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/compressed_model'

## Fine-tuning 모델 압축 (model.tar.gz)

In [29]:
%%writefile shell/finetuned_model_compression_upload.sh

cd merged_model
cp -r ../src/requirements.txt ./
sudo rm -rf code
tar cvf - * | pigz > model.tar.gz

cd ..
mv merged_model/model.tar.gz ./model.tar.gz

Overwriting shell/finetuned_model_compression_upload.sh


In [30]:
%%time
!sh ./shell/finetuned_model_compression_upload.sh

added_tokens.json
config.json
generation_config.json
merges.txt
model-00001-of-00005.safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


model-00002-of-00005.safetensors
model-00003-of-00005.safetensors
model-00004-of-00005.safetensors
model-00005-of-00005.safetensors
model.safetensors.index.json
requirements.txt
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.json
CPU times: user 266 ms, sys: 292 ms, total: 558 ms
Wall time: 42.7 s


In [31]:
!aws s3 cp ./model.tar.gz $compressed_model_path/finetuned/model.tar.gz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


upload: ./model.tar.gz to s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/compressed_model/finetuned/model.tar.gz


## pretrained 모델 압축

In [32]:
!rm -rf $registered_model/original

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
%%writefile shell/pretrained_model_compression_upload.sh

cd qwen3-4b
tar cvf - * | pigz > pretrained_model.tar.gz

cd ..
mv qwen3-4b/pretrained_model.tar.gz ./pretrained_model.tar.gz

Overwriting shell/pretrained_model_compression_upload.sh


In [34]:
%%time
!sh ./shell/pretrained_model_compression_upload.sh
!aws s3 cp ./pretrained_model.tar.gz $compressed_model_path/pretrained/model.tar.gz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


added_tokens.json
config.json
generation_config.json
merges.txt
model-00001-of-00003.safetensors
model-00002-of-00003.safetensors
model-00003-of-00003.safetensors
model.safetensors.index.json
README.md
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


upload: ./pretrained_model.tar.gz to s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/compressed_model/pretrained/model.tar.gz
CPU times: user 665 ms, sys: 881 ms, total: 1.55 s
Wall time: 1min 9s


In [35]:
%store merged_save_dir
%store checkpoint_s3uri
%store compressed_model_path

Stored 'merged_save_dir' (str)
Stored 'checkpoint_s3uri' (str)
Stored 'compressed_model_path' (str)
